## Workshop Homework

### Installing Required Dependencies

In [2]:
!pip install dlt[duckdb]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.7/540.7 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.7 MB/s eta 0:00:00


### Using a Generator

In [3]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n**0.5
        n += 1

In [4]:
sum = 0
generator = square_root_generator(5)

for sqrt in generator:
    sum += sqrt

print(f"The sum of outputs of the generator for limit = 5 is {sum}")

The sum of outputs of the generator for limit = 5 is 8.382332347441762


In [5]:
x = 13
i = 1

generator = square_root_generator(x)
for sqrt in generator:
    if i == 13:
        print(f"The 13th number yielded is {sqrt}")
    i += 1

The 13th number yielded is 3.605551275463989


### Duckdb with dlt

In [6]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)

{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [7]:
import dlt
import duckdb

In [8]:
pipe = dlt.pipeline(destination='duckdb', dataset_name='generators_people')

In [9]:
info = pipe.run(people_1(),
                    table_name="people_table",
					)

In [10]:
print(info)

Pipeline dlt_colab_kernel_launcher load step completed in 0.40 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators_people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708010751.571547 is LOADED and contains no failed jobs


In [11]:
conn = duckdb.connect(f"{pipe.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipe.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

people_df = conn.sql("SELECT * FROM people_table").df()
display(people_df)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_table        │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708010751.571547,7MbF6PGnNdkj/A
1,2,Person_2,27,City_A,1708010751.571547,6XaMAhAhe3DGXg
2,3,Person_3,28,City_A,1708010751.571547,592F61s1M4V/lw
3,4,Person_4,29,City_A,1708010751.571547,AYQJkB0Q4zjETA
4,5,Person_5,30,City_A,1708010751.571547,ZVU7L1r9ytOW3A


In [12]:
pipe = dlt.pipeline(destination='duckdb', dataset_name='generators_people')

In [13]:
info_2 = pipe.run(people_2(),
										table_name="people_table",
										write_disposition="append")

In [14]:
print(info_2)

Pipeline dlt_colab_kernel_launcher load step completed in 0.24 seconds
1 load package(s) were loaded to destination duckdb and into dataset generators_people
The duckdb destination used duckdb:////content/dlt_colab_kernel_launcher.duckdb location to store data
Load package 1708010753.7082837 is LOADED and contains no failed jobs


In [15]:
conn.sql(f"SET search_path = '{pipe.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

people_df = conn.sql("SELECT * FROM people_table").df()
display(people_df)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_table        │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708010751.571547,7MbF6PGnNdkj/A,None
1,2,Person_2,27,City_A,1708010751.571547,6XaMAhAhe3DGXg,None
2,3,Person_3,28,City_A,1708010751.571547,592F61s1M4V/lw,None
3,4,Person_4,29,City_A,1708010751.571547,AYQJkB0Q4zjETA,None
4,5,Person_5,30,City_A,1708010751.571547,ZVU7L1r9ytOW3A,None
5,3,Person_3,33,City_B,1708010753.7082837,YElejSXsKdGocQ,Job_3
6,4,Person_4,34,City_B,1708010753.7082837,WcS9aVSSThVXBw,Job_4
7,5,Person_5,35,City_B,1708010753.7082837,UYYXhztpSb6hFw,Job_5
8,6,Person_6,36,City_B,1708010753.7082837,cFXgNg6T/S7zoQ,Job_6
9,7,Person_7,37,City_B,1708010753.7082837,6AkgDr+GqaXaYw,Job_7


In [16]:
conn.sql("SELECT SUM(age) FROM people_table")

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

#### Merging two tables on Primary Key

In [17]:
pipe_merge = dlt.pipeline(destination='duckdb', dataset_name='generators_people_merge')

In [18]:
info = pipe_merge.run(people_1(),
                    table_name="people_table_merge",
                    primary_key = 'id'
					)

In [19]:
conn = duckdb.connect(f"{pipe_merge.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipe_merge.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people_table_merge below:")

people_df = conn.sql("SELECT * FROM people_table_merge").df()
display(people_df)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_table_merge  │
└─────────────────────┘




 people_table_merge below:


,id,name,age,city,_dlt_load_id,_dlt_id
0,1,Person_1,26,City_A,1708010881.322861,NYtjp7pNMhU+Dg
1,2,Person_2,27,City_A,1708010881.322861,lUPXuq+0eWlHXg
2,3,Person_3,28,City_A,1708010881.322861,tSU1gMwR3MZSEg
3,4,Person_4,29,City_A,1708010881.322861,L2JAIcMaXo4SLQ
4,5,Person_5,30,City_A,1708010881.322861,o9TwY8w8/likPg


In [20]:
info_2 = pipe_merge.run(people_2(),
										table_name="people_table_merge",
										write_disposition="merge",
                                        primary_key='id')

In [21]:
conn = duckdb.connect(f"{pipe_merge.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipe_merge.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people_table_merge below:")

people_df = conn.sql("SELECT * FROM people_table_merge").df()
display(people_df)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people_table_merge  │
└─────────────────────┘




 people_table_merge below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708010881.322861,NYtjp7pNMhU+Dg,None
1,2,Person_2,27,City_A,1708010881.322861,lUPXuq+0eWlHXg,None
2,8,Person_8,38,City_B,1708010985.8541877,SOMuqHywGrfcMQ,Job_8
3,4,Person_4,34,City_B,1708010985.8541877,3F+/PGf/380MKg,Job_4
4,5,Person_5,35,City_B,1708010985.8541877,0eijF21IIXzeUw,Job_5
5,7,Person_7,37,City_B,1708010985.8541877,Zi6aKD2ozApc1g,Job_7
6,3,Person_3,33,City_B,1708010985.8541877,nmjxGgaNSlaPDA,Job_3
7,6,Person_6,36,City_B,1708010985.8541877,wMJ0u+ajr7Jdgw,Job_6


In [22]:
conn.sql("SELECT SUM(age) FROM people_table_merge")

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      266 │
└──────────┘